In [1]:
from webscrape_fast import *
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm

c:\Users\mr\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('data/cities_urls.json') as f:
    cities_urls = json.load(f)
print(len(cities_urls))

304279


In [56]:
def get_all_cities_data(cities_data=[], batch=(0, 0)):
    if not cities_data:
        for _ in cities_urls:
            cities_data.append("not_loaded")

    if batch[1] == 0:
        batch[1] = len(cities_data())

    executor = ThreadPoolExecutor()
    futures = [(batch[0] + i, executor.submit(get_city_data, cities_urls[batch[0] + i])) for i, _ in filter(lambda x: isinstance(x[1], str) and x[1] != "<class 'ValueError'>", enumerate(cities_data[batch[0]:batch[1]]))]
    for i, f in tqdm(futures):
        try:
            cities_data[i] = f.result()
        except Exception as exc:
            cities_data[i] = str(type(exc))
    return cities_data

In [4]:
cities_data = []

In [46]:
cities_data = get_all_cities_data(cities_data, (0, 1000))

100%|██████████| 1598/1598 [04:56<00:00,  5.38it/s]


In [55]:
# print stats
errored = []
loaded = 0
for i, el in enumerate(cities_data[0:10000]):
    if not isinstance(el, str):
        loaded += 1
    elif True: # el != "not_loaded" and el != "<class 'ValueError'>":
        errored.append((i, el, cities_urls[i]))
print(len(errored))
for el in errored:
    print(el)
print("loaded {}/{}".format(loaded, len(cities_data)))

print(set([el for _, el, _ in errored]))

1695
(469, "<class 'ValueError'>", 'https://tcktcktck.org/afghanistan/kandahar/chenartu')
(831, "<class 'ValueError'>", 'https://tcktcktck.org/afghanistan/sar-e-pul/juy-emam')
(849, "<class 'ValueError'>", 'https://tcktcktck.org/afghanistan/sar-e-pul/khaneh-ye-hajji-mohammad')
(893, "<class 'ValueError'>", 'https://tcktcktck.org/afghanistan/uruzgan/deh-e-rawut')
(908, "<class 'ValueError'>", 'https://tcktcktck.org/afghanistan/uruzgan/kash')
(914, "<class 'ValueError'>", 'https://tcktcktck.org/afghanistan/uruzgan/kend')
(934, "<class 'ValueError'>", 'https://tcktcktck.org/afghanistan/uruzgan/khoshko-kharab')
(940, "<class 'requests.exceptions.ConnectTimeout'>", 'https://tcktcktck.org/afghanistan/uruzgan/khvoshkhuy')
(941, "<class 'requests.exceptions.ConnectTimeout'>", 'https://tcktcktck.org/afghanistan/uruzgan/kibay')
(942, "<class 'requests.exceptions.ConnectTimeout'>", 'https://tcktcktck.org/afghanistan/uruzgan/kohneh-qal-eh')
(943, "<class 'requests.exceptions.ConnectTimeout'>", 'ht

In [52]:
cities_data_to_json = []
for el in cities_data:
    if isinstance(el, str):
        cities_data_to_json.append(el)
    else:
        cities_data_to_json.append((el[0], json.loads(el[1].to_json())))
    
with open('data/cities_data.json', 'w') as f:
    json.dump(cities_data_to_json, f, indent=4)

In [61]:
with open('data/cities_data.json') as f:
    cities_data_loaded_raw = json.load(f)
    cities_data_loaded = []
    for el in cities_data_loaded_raw:
        if isinstance(el, str):
            cities_data_loaded.append(el)
        else:
            cities_data_loaded.append((el[0], pd.DataFrame.from_dict(el[1])))
print(len(cities_data_loaded))

304279


In [63]:
print(cities_data_loaded[2000])

({'city': 'Aqkowtal', 'region': 'Baghlan', 'country': 'Afghanistan', 'coords': [68.479103, 35.970844]},                                    Month    Jan    Feb    Mar    Apr    May  \
0                    Record high °C (°F)  17.48  23.31  32.05  34.00  34.97   
1                   Average high °C (°F)   7.76   8.12  15.53  20.51  26.25   
2                     Daily mean °C (°F)   4.48   5.16  12.40  17.37  23.27   
3                    Average low °C (°F)   0.18   0.16   6.69  10.85  16.38   
4                     Record low °C (°F) -10.68 -21.37 -11.66  -0.97   7.77   
5      Average precipitation mm (inches)  19.84  30.56  29.28  41.09  25.87   
6  Average precipitation days (≥ 1.0 mm)   4.15   6.36   6.45   8.21   6.18   
7          Average relative humidity (%)  51.64  59.94  50.60  47.71  34.34   
8            Mean monthly sunshine hours   7.46   7.38   9.99  12.29  13.50   

     Jun    Jul    Aug    Sep    Nov    Oct    Dec   Year  
0  40.79  39.82  38.85  34.00  30.11  27.20  